In [1]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import os
import pandas as pd
import numpy as np
from PIL import Image
import CONST
from utils import (
    filter_empty_columns,
    num_duplicated_values,
    load_onnx,
    init_session_onnx,
    find_path,
    load_image,
    forward_transform,
    get_segmentation,
    resize_segmentation,
    remove_image_extension,
)

# Annotation cleaning

In [3]:
# Check annotation dataframe
df = pd.read_csv(CONST.ANNOTATION_PATH)
df

,Unnamed: 0,WoundDetail.Id,ImageDetail.Id,ImageDetail.ImageFilename,ImageDetail.Folder,ImageDetail.PatientId,ImageDetail.LastAmended,ImageDetail.AmendedBy,ImageDetail.ReviewedBy,ImageDetail.Status,...,WoundDetail.AreaOfInterestHeight,Unnamed: 38,image_path,overlay_path,folder,ImageDetail.MachineName,ImageDetail.PriorMinorAmputation,WoundDetail.WoundBedUnfilled,WoundDetail.Unfilled,Unnamed: 25
0,0,7,9,VWI11962 2021-05-05-10-26-05.jpg,C:\Users\night\Desktop\TTSH\Research\AITIS\VWI...,VWI11962,12/7/2021 14:31,Audrey Tan,NaN,ReadyForReview,...,1047,NaN,230921 export_a\Raw images\VWI11962 2021-05-05...,230921 export_a\Overlays\Ov_VWI11962 2021-05-0...,230921 export_a,NaN,NaN,NaN,NaN,NaN
1,1,8,10,VWI11961 2021-05-05-07-51-40.jpg,C:\Users\night\Desktop\TTSH\Research\AITIS\VWI...,VWI11961,12/7/2021 17:03,Audrey Tan,NaN,ReadyForReview,...,840,NaN,230921 export_a\Raw images\VWI11961 2021-05-05...,230921 export_a\Overlays\Ov_VWI11961 2021-05-0...,230921 export_a,NaN,NaN,NaN,NaN,NaN
2,2,9,12,VWI11959 2021-05-04-12-14-56.jpg,C:\Users\night\Desktop\TTSH\Research\AITIS\VWI...,VWI11959,14/7/2021 0:53,Audrey Tan,NaN,ReadyForReview,...,685,NaN,230921 export_a\Raw images\VWI11959 2021-05-04...,230921 export_a\Overlays\Ov_VWI11959 2021-05-0...,230921 export_a,NaN,NaN,NaN,NaN,NaN
3,3,10,15,VWI11953 2021-05-04-08-17-12.jpg,C:\Users\night\Desktop\TTSH\Research\AITIS\VWI...,VWI11953,14/7/2021 1:02,Audrey Tan,NaN,ReadyForReview,...,708,NaN,230921 export_a\Raw images\VWI11953 2021-05-04...,230921 export_a\Overlays\Ov_VWI11953 2021-05-0...,230921 export_a,NaN,NaN,NaN,NaN,NaN
4,4,11,18,VWI11950 2021-05-03-12-59-56.jpg,C:\Users\night\Desktop\TTSH\Research\AITIS\VWI...,VWI11950,14/7/2021 9:02,Audrey Tan,NaN,ReadyForReview,...,1600,NaN,230921 export_a\Raw images\VWI11950 2021-05-03...,230921 export_a\Overlays\Ov_VWI11950 2021-05-0...,230921 export_a,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2718,156,167,161,027 V5_06.02.2020 HEALED.JPG,C:\Users\chanx\Desktop\J\Meta-data\KM\Images\027,27,16/11/2021 13:13,NaN,NaN,ReadyForReview,...,1190,NaN,Meta KM 014 to 029\Raw images\027\027 V5_06.02...,Meta KM 014 to 029\Overlays\Ov_027 V5_06.02.20...,Meta KM 014 to 029,NaN,NaN,NaN,NaN,NaN
2719,157,168,162,WA028_22.07.20_V4.JPG,C:\Users\chanx\Desktop\J\Meta-data\KM\Images\028,WA028_22,16/11/2021 13:13,NaN,NaN,ReadyForReview,...,783,NaN,Meta KM 014 to 029\Raw images\028\WA028_22.07....,Meta KM 014 to 029\Overlays\Ov_WA028_22.07.20_...,Meta KM 014 to 029,NaN,NaN,NaN,NaN,NaN
2720,158,169,163,WA028_29.07.20_V5.JPG,C:\Users\chanx\Desktop\J\Meta-data\KM\Images\028,WA028_29,16/11/2021 13:14,NaN,NaN,ReadyForReview,...,795,NaN,Meta KM 014 to 029\Raw images\028\WA028_29.07....,Meta KM 014 to 029\Overlays\Ov_WA028_29.07.20_...,Meta KM 014 to 029,NaN,NaN,NaN,NaN,NaN
2721,159,170,164,WA029_07.08.20_V4.JPG,C:\Users\chanx\Desktop\J\Meta-data\KM\Images\029,WA029_07,26/10/2021 12:23,NaN,NaN,ReadyForReview,...,1298,NaN,Meta KM 014 to 029\Raw images\029\WA029_07.08....,Meta KM 014 to 029\Overlays\Ov_WA029_07.08.20_...,Meta KM 014 to 029,NaN,NaN,NaN,NaN,NaN


In [4]:
# Filter the data into only selected column
df = df.filter(items=CONST.SELECT_COLUMN)

# If the wound type and wound location is empty, we will drop the data
df = filter_empty_columns(df, CONST.WOUND_TYPE)
df = filter_empty_columns(df, CONST.WOUND_LOCATION)

# There are many duplications on the file name that we need to drop them
df = df.drop_duplicates()
df = df.drop_duplicates(subset=[CONST.FILE_NAME], keep="first")

In [5]:
# Re-check to see if there is still any duplicated file name
print(f"Number of overlapped file: {num_duplicated_values(df, CONST.FILE_NAME)}")

Number of overlapped file: 0


In [6]:
# Check the data again
df

,ImageDetail.ImageFilename,ImageDetail.hasRuler,ImageDetail.WoundType,ImageDetail.OtherWoundBedCategory,WoundDetail.WoundDepth,WoundDetail.Location,ImageDetail.AreaOfInterestX,ImageDetail.AreaOfInterestWidth,ImageDetail.AreaOfInterestY,ImageDetail.AreaOfInterestHeight
0,VWI11962 2021-05-05-10-26-05.jpg,False,Neuro-Ischemic Ulcers (NIU),NaN,Deep,Left Dorsal Metatarsals;,55,1145,300,1047
1,VWI11961 2021-05-05-07-51-40.jpg,False,Neuro-Ischemic Ulcers (NIU),Exposed Tendon/Bone,Deep,Right Plantar Calcaneus;,36,1021,498,840
2,VWI11959 2021-05-04-12-14-56.jpg,False,Neuro-Ischemic Ulcers (NIU),NaN,Deep,Right Medial Metatarsals;,107,744,194,685
3,VWI11953 2021-05-04-08-17-12.jpg,False,Neuro-Ischemic Ulcers (NIU),NaN,Deep,Left Plantar Calcaneus;,22,648,320,708
4,VWI11950 2021-05-03-12-59-56.jpg,False,Neuro-Ischemic Ulcers (NIU),NaN,Deep,Left Dorsal Metatarsals;,0,1200,0,1600
...,...,...,...,...,...,...,...,...,...,...
2716,027 V3_08.01.2020.JPG,True,Venous Leg Ulcer (VLU),NaN,Epidermal Layer,Left Shin;,944,673,49,1136
2717,027 V4_30.01.2020.JPG,True,Venous Leg Ulcer (VLU),NaN,Epidermal Layer,Left Shin;,1357,619,95,1080
2718,027 V5_06.02.2020 HEALED.JPG,True,Venous Leg Ulcer (VLU),NaN,Epidermal Layer,Left Shin;,1048,672,88,1190
2719,WA028_22.07.20_V4.JPG,False,Venous Leg Ulcer (VLU),NaN,Epidermal Layer,Left Medial Ankle;,640,1003,919,783


# Annotation preprocessing

As there are many data with duplicated semantics (for example: a same wound type in a same location but different shape will cause the model confuse of what to render at each pixel), therefore, we will need to tidy up the dataset so that:
- Different sample will have different semantics
- This will make the model less confusing while generating the images

In [7]:
# Check for duplicated semantics
df_duplicated = df.duplicated(
    subset=[
        CONST.WOUND_RULER,
        CONST.WOUND_TYPE,
        CONST.WOUND_BED,
        CONST.WOUND_DEPTH,
        CONST.WOUND_LOCATION,
    ],
    keep="last",
)
df_cleaned = df[df_duplicated == False]
df_cleaned

,ImageDetail.ImageFilename,ImageDetail.hasRuler,ImageDetail.WoundType,ImageDetail.OtherWoundBedCategory,WoundDetail.WoundDepth,WoundDetail.Location,ImageDetail.AreaOfInterestX,ImageDetail.AreaOfInterestWidth,ImageDetail.AreaOfInterestY,ImageDetail.AreaOfInterestHeight
1,VWI11961 2021-05-05-07-51-40.jpg,False,Neuro-Ischemic Ulcers (NIU),Exposed Tendon/Bone,Deep,Right Plantar Calcaneus;,36,1021,498,840
5,VWI11949 2021-05-03-08-46-12.jpg,False,Surgical Site Infections (SSI),NaN,Deep,Right Medial Calf;,0,1600,0,1200
8,VWI11946 2021-05-03-08-08-53.jpg,False,Neuro-Ischemic Ulcers (NIU),NaN,Deep,Right Medial Calf;,0,1600,0,1200
14,VWI11802 2021-04-01-08-40-23.jpg,False,Venous Leg Ulcer (VLU),NaN,Dermal Layer,Right Medial Calf;Right Medial Malleolus;,102,805,342,942
16,VWI11805 2021-04-01-09-39-22.jpg,False,Neuro-Ischemic Ulcers (NIU),NaN,Deep,Right Dorsal Metatarsals;Right Plantar Metatar...,294,856,94,1400
...,...,...,...,...,...,...,...,...,...,...
2704,024 V5_13.04.20.JPG,True,Venous Leg Ulcer (VLU),NaN,Epidermal Layer,Left Lateral Ankle;,522,904,463,672
2713,026 V4_03.02.2020.JPG,True,Venous Leg Ulcer (VLU),NaN,Epidermal Layer,Left Medial Calf;,1222,562,428,655
2718,027 V5_06.02.2020 HEALED.JPG,True,Venous Leg Ulcer (VLU),NaN,Epidermal Layer,Left Shin;,1048,672,88,1190
2719,WA028_22.07.20_V4.JPG,False,Venous Leg Ulcer (VLU),NaN,Epidermal Layer,Left Medial Ankle;,640,1003,919,783


In [8]:
# Save the processed annotation file
df_cleaned.to_csv(CONST.ANNOTATION_PROCESSED_PATH, index=False)

# Sample segmentation

In [9]:
# Read the processed annotation
annotation = pd.read_csv(CONST.ANNOTATION_PROCESSED_PATH)
annotation

,ImageDetail.ImageFilename,ImageDetail.hasRuler,ImageDetail.WoundType,ImageDetail.OtherWoundBedCategory,WoundDetail.WoundDepth,WoundDetail.Location,ImageDetail.AreaOfInterestX,ImageDetail.AreaOfInterestWidth,ImageDetail.AreaOfInterestY,ImageDetail.AreaOfInterestHeight
0,VWI11961 2021-05-05-07-51-40.jpg,False,Neuro-Ischemic Ulcers (NIU),Exposed Tendon/Bone,Deep,Right Plantar Calcaneus;,36,1021,498,840
1,VWI11949 2021-05-03-08-46-12.jpg,False,Surgical Site Infections (SSI),NaN,Deep,Right Medial Calf;,0,1600,0,1200
2,VWI11946 2021-05-03-08-08-53.jpg,False,Neuro-Ischemic Ulcers (NIU),NaN,Deep,Right Medial Calf;,0,1600,0,1200
3,VWI11802 2021-04-01-08-40-23.jpg,False,Venous Leg Ulcer (VLU),NaN,Dermal Layer,Right Medial Calf;Right Medial Malleolus;,102,805,342,942
4,VWI11805 2021-04-01-09-39-22.jpg,False,Neuro-Ischemic Ulcers (NIU),NaN,Deep,Right Dorsal Metatarsals;Right Plantar Metatar...,294,856,94,1400
...,...,...,...,...,...,...,...,...,...,...
506,024 V5_13.04.20.JPG,True,Venous Leg Ulcer (VLU),NaN,Epidermal Layer,Left Lateral Ankle;,522,904,463,672
507,026 V4_03.02.2020.JPG,True,Venous Leg Ulcer (VLU),NaN,Epidermal Layer,Left Medial Calf;,1222,562,428,655
508,027 V5_06.02.2020 HEALED.JPG,True,Venous Leg Ulcer (VLU),NaN,Epidermal Layer,Left Shin;,1048,672,88,1190
509,WA028_22.07.20_V4.JPG,False,Venous Leg Ulcer (VLU),NaN,Epidermal Layer,Left Medial Ankle;,640,1003,919,783


In [10]:
# Initialise the model
model = load_onnx(CONST.SEG_MODEl_PATH)
session = init_session_onnx(model)

In [11]:
# For each record, load file
for _, row in annotation.iterrows():
    # Get file path
    file_path = find_path(
        filename=row[CONST.FILE_NAME], root_dir=CONST.UNPROCESSED_IMAGES_DIR
    )

    # Skip the missing file
    if file_path is None:
        continue

    # Crop the ROI
    if row[CONST.ROI_Y_HEIGHT] != 0 and row[CONST.ROI_X_WIDTH] != 0:
        image = load_image(file_path)[
            row[CONST.ROI_Y] : (row[CONST.ROI_Y] + row[CONST.ROI_Y_HEIGHT]),
            row[CONST.ROI_X] : (row[CONST.ROI_X] + row[CONST.ROI_X_WIDTH]),
        ]
    else:
        image = load_image(file_path)

    # Prepare for tensor
    model_input = np.expand_dims(
        forward_transform(
            image=image, target_size=CONST.SEG_INPUT_SIZE, to_tensor=False
        ),
        axis=0,
    ).transpose((0, 3, 1, 2))

    # Get segments
    segment = get_segmentation(_sample=model_input, _session=session)
    segment = resize_segmentation(
        _segmentation_matrix=segment, _size=image.shape[:-1][::-1]
    )

    # Save images and segments
    Image.fromarray(image).save(
        os.path.join(CONST.PROCESSED_IMAGES_DIR, row[CONST.FILE_NAME])
    )
    np.save(
        os.path.join(
            CONST.PROCESSED_SEGMENT_DIR,
            remove_image_extension(row[CONST.FILE_NAME]),
        )
        + ".npy",
        arr=segment,
    )

print("Done segmentation!")

Done segmentation!
